In [ ]:
"""
Implementation of a simple multivalent binding model.
"""

import numpy as np
import pandas as pd
from scipy.optimize import minimize
from src.MBmodel import cytBindingModel, SigData

In [ ]:
def fitFunc():
    "Runs least squares fitting for various model parameters, and returns the minimizers"
    x0 = np.array([-12, 1.0, 8.6, 8, 8, 7.6, 7, 7, 9.08, 8, 8, 8.59, 7, 7])  # KXSTAR, slopeT1h, slopeT1m, mIL4-IL4Ra, mIL4-Gamma, mIL4-IL13Ra, mNeo4-IL4Ra, mNeo4-Gamma, mNeo4-IL13Ra, hIL4-IL4Ra, hIL4-Gamma, hIL4-IL13Ra, hNeo4-IL4Ra, hNeo4-Gamma, hNeo4-IL13Ra (Log 10)
    bnds = ((-14, -10), (0, 10), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11))
    parampredicts = minimize(resids, x0, method='L-BFGS-B', bounds=bnds, options={"disp": 999})
    assert parampredicts.success
    return parampredicts.x

def resids(x, retDF=False):
    """"Returns residuals against signaling data"""
    masterSTAT = pd.DataFrame(columns={"Cell", "Ligand", "Concentration", "Animal", "Experimental", "Predicted"})
    Kx = x[0]
    relRecp = x[1]
    xPow = np.power(10, x)
    CplxDict = {"mIL4": [[xPow[2], 1e2, 1e2], [1e2, xPow[3], xPow[4]]],
    "mNeo4": [[xPow[5], 1e2, 1e2], [1e2, xPow[6], xPow[7]]],
    "hIL4": [[xPow[8], 1e2, 1e2], [1e2, xPow[9], xPow[10]]],
    "hNeo4": [[xPow[11], 1e2, 1e2], [1e2, xPow[12], xPow[13]]]}

    for cell in SigData.Cell.unique():
        for animal in SigData.loc[SigData.Cell == cell].Animal.unique():
            for ligand in SigData.loc[(SigData.Cell == cell) & (SigData.Animal == animal)].Ligand.unique():
                isoData = SigData.loc[(SigData.Cell == cell) & (SigData.Animal == animal) & (SigData.Ligand == ligand)]
                Concs = isoData.Concentration.values
                normSigs = isoData.Signal.values
                ligCplx = CplxDict[ligand]
                if animal == "Human":
                    results = cytBindingModel(Kx, ligCplx, Concs, cell, animal, relRecp)
                else:
                    results = cytBindingModel(Kx, ligCplx, Concs, cell, animal, relRecp)
                masterSTAT = masterSTAT.append(pd.DataFrame({"Cell": cell, "Ligand": ligand, "Concentration": Concs, "Animal": animal, "Experimental": normSigs, "Predicted": results}))
            
            # Normalize
            masterSTAT.loc[(masterSTAT.Cell == cell) & (masterSTAT.Animal == animal), "Predicted"] /= masterSTAT.loc[(masterSTAT.Cell == cell) & (masterSTAT.Animal == animal)].Predicted.max()
            masterSTAT.loc[(masterSTAT.Cell == cell) & (masterSTAT.Animal == animal), "Experimental"] /= masterSTAT.loc[(masterSTAT.Cell == cell) & (masterSTAT.Animal == animal)].Experimental.max()
 
    if retDF:
        return masterSTAT
    else:
        print(np.linalg.norm(masterSTAT.Predicted.values - masterSTAT.Experimental.values))
        print(x)
        return np.linalg.norm(masterSTAT.Predicted.values - masterSTAT.Experimental.values)

In [ ]:
fitFunc()